# Análise exploratória de dados

In [ ]:
### Import de bibliotecas para manipulação dos dados 
import pandas as pd
import numpy as np
import math

### para visualizacao 
import seaborn as sns
import matplotlib.pyplot as plt

### para obter estatisticas de teste
import scipy.stats as stats
from scipy.stats import norm
from sklearn.preprocessing import LabelEncoder

: 

## 1. Descrição dos dados

In [ ]:
df = pd.read_csv('../data/raw/Orange_Quality_Data.csv')

##### O dataset "Orange Quality Data.csv" contém 241 linhas e 11 colunas, e ele foi retirado dos datasets da kaggle. 
##### As variáveis incluem: 
**Size (cm)** 
**Weight (g)** 
**Brix (Sweetness)**
**pH (Acidity)**
**Softness (1-5)**
**HarvestTime (days)** 
**Ripeness (1-5)**
**Color**
**Variety**
**Blemishes (Y/N)**
**Quality (1-5).**

* **Size (cm):** Diâmetro da laranja em centímetros, representando o tamanho da fruta.
* **Weight (g):** Peso da laranja em gramas, indicando a massa da fruta.
* **Brix (Sweetness):** Nível de açúcar da laranja, medido pela escala Brix, que indica o grau de doçura.
* **pH (Acidity):** Nível de acidez da laranja, medido pelo pH, que indica o sabor azedo da fruta.
* **Softness (1-5):** Nível de maciez da laranja em uma escala de 1 a 5, onde 1 indica uma fruta muito dura e 5 indica uma fruta muito macia.
* **HarvestTime (days):** Número de dias desde o plantio até a colheita, indicando o tempo de cultivo.
* **Ripeness (1-5):** Nível de maturação da laranja em uma escala de 1 a 5, onde 1 indica uma fruta verde e 5 indica uma fruta madura.
* **Color:** Cor da casca da laranja, indicando a variedade e o estágio de maturação.
* **Variety:** Variedade da laranja, como Valencia, Navel, Cara Cara, etc.
* **Blemishes (Y/N):** Indica se a laranja possui manchas ou defeitos visíveis (Sim ou Não).
* **Quality (1-5):** Qualidade geral da laranja em uma escala de 1 a 5, avaliando diversos atributos como tamanho, aparência, sabor e textura.

In [ ]:
def  separador():
    print("=" * 50 + " Separador " + "=" * 50)
separador()

#Análise das 10 primeiras linhas 
df.head(10)

================================================== Separador ==================================================


,Size (cm),Weight (g),Brix (Sweetness),pH (Acidity),Softness (1-5),HarvestTime (days),Ripeness (1-5),Color,Variety,Blemishes (Y/N),Quality (1-5)
0,7.5,180,12.0,3.2,2.0,10,4.0,Orange,Valencia,N,4.0
1,8.2,220,10.5,3.4,3.0,14,4.5,Deep Orange,Navel,N,4.5
2,6.8,150,14.0,3.0,1.0,7,5.0,Light Orange,Cara Cara,N,5.0
3,9.0,250,8.5,3.8,4.0,21,3.5,Orange-Red,Blood Orange,N,3.5
4,8.5,210,11.5,3.3,2.5,12,5.0,Orange,Hamlin,Y (Minor),4.5


In [ ]:
separador()

# Exibindo as informações do DataFrame
df.info()

================================================== Separador ==================================================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Size (cm)           241 non-null    float64
 1   Weight (g)          241 non-null    int64  
 2   Brix (Sweetness)    241 non-null    float64
 3   pH (Acidity)        241 non-null    float64
 4   Softness (1-5)      241 non-null    float64
 5   HarvestTime (days)  241 non-null    int64  
 6   Ripeness (1-5)      241 non-null    float64
 7   Color               241 non-null    object 
 8   Variety             241 non-null    object 
 9   Blemishes (Y/N)     241 non-null    object 
 10  Quality (1-5)       241 non-null    float64
dtypes: float64(6), int64(2), object(3)
memory usage: 20.8+ KB


In [ ]:
separador()
#Resumo estatístico
df.describe()

================================================== Separador ==================================================


,Size (cm),Weight (g),Brix (Sweetness),pH (Acidity),Softness (1-5),HarvestTime (days),Ripeness (1-5),Quality (1-5)
count,241.000000,241.000000,241.000000,241.000000,241.000000,241.000000,241.000000,241.000000
mean,7.844813,205.128631,10.907884,3.473900,3.072614,15.344398,3.599585,3.817427
std,1.086002,56.461012,2.760446,0.421007,1.323630,5.323852,1.205214,1.014410
min,6.000000,100.000000,5.500000,2.800000,1.000000,4.000000,1.000000,1.000000
25%,6.900000,155.000000,8.500000,3.200000,2.000000,11.000000,3.000000,3.000000
50%,7.800000,205.000000,11.000000,3.400000,3.000000,15.000000,4.000000,4.000000
75%,8.700000,252.000000,13.400000,3.800000,4.000000,20.000000,4.500000,4.500000
max,10.000000,300.000000,16.000000,4.400000,5.000000,25.000000,5.000000,5.000000


In [22]:
dicionario_dados = {
    "Size (cm)": {"descricao": "Diâmetro da laranja em centímetros", "tipo": "quantitativa", "subtipo": "contínua"},
    "Weight (g)": {"descricao": "Peso da laranja em gramas", "tipo": "quantitativa", "subtipo": "contínua"},
    "Brix (Sweetness)": {"descricao": "Nível de açúcar da laranja (Brix)", "tipo": "quantitativa", "subtipo": "contínua"},
    "pH (Acidity)": {"descricao": "Nível de acidez da laranja (pH)", "tipo": "quantitativa", "subtipo": "contínua"},
    "Softness (1-5)": {"descricao": "Nível de maciez da laranja em uma escala de 1 a 5", "tipo": "quantitativa", "subtipo": "discreta"},
    "HarvestTime (days)": {"descricao": "Número de dias desde o plantio até a colheita", "tipo": "quantitativa", "subtipo": "discreta"},
    "Ripeness (1-5)": {"descricao": "Nível de maturação da laranja em uma escala de 1 a 5", "tipo": "quantitativa", "subtipo": "discreta"},
    "Color": {"descricao": "Cor da casca da laranja", "tipo": "qualitativa", "subtipo": "nominal"},
    "Variety": {"descricao": "Variedade da laranja", "tipo": "qualitativa", "subtipo": "nominal"},
    "Blemishes (Y/N)": {"descricao": "Presença de manchas na laranja (Sim/Não)", "tipo": "qualitativa", "subtipo": "binária"},
    "Quality (1-5)": {"descricao": "Qualidade geral da laranja em uma escala de 1 a 5", "tipo": "quantitativa", "subtipo": "discreta"}
}

separador()
print(dicionario_dados)

================================================== Separador ==================================================
{'Size (cm)': {'descricao': 'Diâmetro da laranja em centímetros', 'tipo': 'quantitativa', 'subtipo': 'contínua'}, 'Weight (g)': {'descricao': 'Peso da laranja em gramas', 'tipo': 'quantitativa', 'subtipo': 'contínua'}, 'Brix (Sweetness)': {'descricao': 'Nível de açúcar da laranja (Brix)', 'tipo': 'quantitativa', 'subtipo': 'contínua'}, 'pH (Acidity)': {'descricao': 'Nível de acidez da laranja (pH)', 'tipo': 'quantitativa', 'subtipo': 'contínua'}, 'Softness (1-5)': {'descricao': 'Nível de maciez da laranja em uma escala de 1 a 5', 'tipo': 'quantitativa', 'subtipo': 'discreta'}, 'HarvestTime (days)': {'descricao': 'Número de dias desde o plantio até a colheita', 'tipo': 'quantitativa', 'subtipo': 'discreta'}, 'Ripeness (1-5)': {'descricao': 'Nível de maturação da laranja em uma escala de 1 a 5', 'tipo': 'quantitativa', 'subtipo': 'discreta'}, 'Color': {'descricao': 'Cor da casc

## 2. Perguntas de partida e hipóteses

# a: Como a maciez das laranjas varia com o tempo de colheita e o estágio de maturação? Laranjas mais maduras são sempre mais macias?

In [ ]:
#Coloque seu código aqui 

# b: A cor das laranjas está relacionada à sua qualidade? Quais tonalidades são indicativas de maior qualidade?

In [ ]:
# Análise Exploratória da Relação entre Cor e Qualidade
# Calculando a média da qualidade para cada tonalidade de cor
color_quality = df.groupby('Color')['Quality (1-5)'].mean().sort_values()

# Exibindo a tabela de médias de qualidade por cor para melhor visualização 
print("Qualidade média por tonalidade de cor:")
print(color_quality)

# Gráfico de Barra Qualidade vs Tonalidade de Cor
plt.figure(figsize=(12, 8))
sns.barplot(x=color_quality.index, y=color_quality.values, palette="plasma")

# Configurações do Gráfico
plt.title('Qualidade Média das Laranjas por Tonalidade de Cor')
plt.xlabel('Tonalidade de Cor')
plt.ylabel('Qualidade Média')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Adicionando os valores em cima das barras
for index, value in enumerate(color_quality.values):
    plt.text(index, value + 0.05, f"{value:.2f}", ha='center', va='bottom', fontsize=10)


# Exibir o gráfico
plt.show()

#Espaçamento para melhor visualização 
print(' ')
print(' ')
print(' ')
print(' ')

# Transformando a coluna 'color' em valores numéricos para utilizar a correlação
label_encoder = LabelEncoder()
df['color_encoded'] = label_encoder.fit_transform(df['Color'])

# Cálculo da correlação entre cor e qualidade
correlation = df[['color_encoded', 'Quality (1-5)']].corr().iloc[0, 1]
print(f"Correlação entre cor e qualidade: {correlation:.2f}")

# Gráfico de dispersão
plt.figure(figsize=(10, 6))
sns.scatterplot(x='color_encoded', y='Quality (1-5)', data=df, palette="plasma", hue='Color', legend=False)

# Configurações do gráfico de dispersão
plt.title('Correlação entre Cor das Laranjas e Qualidade')
plt.xlabel('Cor (Codificada)')
plt.ylabel('Qualidade')
plt.grid(True, linestyle='--', alpha=0.6)

# Exibir o gráfico
plt.show()

# Exibir cores originais, codificadas e qualidade em tabela para melhor entendimento
color_mapping = pd.DataFrame({
    'Original Color': label_encoder.classes_,  # Cores originais
    'Encoded Color': range(len(label_encoder.classes_)),  # Cores codificadas
    'Quality': [df[df['Color'] == color]['Quality (1-5)'].mean() for color in label_encoder.classes_]  # Média da qualidade por cor
})

print("\nTabela de Cores e Qualidade:")
print(color_mapping)

# A correlação calculada é de -0.12.
# Interpretação dos Resultados
# Qualidade Média: A análise mostra que a tonalidade Deep Orange apresenta a maior qualidade média (4.31), seguida por Yellow-Orange (4.22). Isso indica que as laranjas dessas cores tendem a ser avaliadas como de maior qualidade. Tonalidades Menores: As tonalidades Light Orange, Orange, e Orange-Red têm médias de qualidade inferiores (3.28 a 3.75), sugerindo que essas cores estão associadas a uma qualidade menor em comparação às tonalidades mais escuras.
# Correlação Negativa: A correlação de -0.12 indica uma relação fraca e negativa entre a cor das laranjas e sua qualidade. Isso sugere que, embora existam variações na qualidade média entre as diferentes tonalidades, não há uma relação forte que indique que a cor mais intensa sempre resulta em maior qualidade.


# c: Qual é a influência das manchas na avaliação da qualidade das laranjas?

In [ ]:
#Coloque seu código aqui 

# d: Como o tempo de colheita afeta as características das laranjas? Existem períodos ideais para a colheita que maximizam a qualidade?

In [ ]:
#Coloque seu código aqui 

# e: A presença de manchas afeta o pH ou a doçura das laranjas? Isso poderia indicar alguma relação entre defeitos visuais e características internas da fruta.

In [ ]:
#Coloque seu código aqui 

## 3. Insights